In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)
from google.cloud import bigquery
from plotly import graph_objs as go

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns

from fintrans_toolbox.src import adjustments as adj
from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t

In [ ]:
client = bigquery.Client()

# SPOC

In [ ]:
df = t.read_spend_origin_and_channel(client, time_period='Month', merchant_channel = 'Face to Face', 
                                     cardholder_origin = 'UNITED KINGDOM', destination_country = '"UNITED KINGDOM"')

In [ ]:
spoc_adj = adj.adjusted_all_spoc(df, time_period='Month', merchant_channel = "Face to Face")

# RPHST

In [ ]:
df_rphst = t.read_retail_performance_high_streets_towns(client, time_period='Month')

In [ ]:
rphst_adj = adj.adjusted_all_rphst(df_rphst, time_period = "Month")

# SML

In [ ]:
df_sml = t.read_spend_merchant_location(client, time_period='Month', cardholder_issuing_level ='Domestic')

In [ ]:
sml_adj = adj.adjusted_all_sml(df_sml, time_period = "Month", cardholder_issuing_level ='Domestic')

# Comparison

In [ ]:
print(rphst_adj['spend'].sum() == df_rphst['spend'].sum())
print(rphst_adj['transactions'].sum() == df_rphst['transactions'].sum())

In [ ]:
print(rphst_adj['spend'].sum() == spoc_adj['spend'].sum())
print(rphst_adj['transactions'].sum() == spoc_adj['transactions'].sum())

In [ ]:
print(sml_adj['spend'].sum() == sml_adj['spend'].sum())
print(sml_adj['transactions'].sum() == sml_adj['transactions'].sum())

In [ ]:
print(rphst_adj['idx_spend'].sum() == spoc_adj['idx_spend'].sum() == sml_adj['idx_spend'].sum())
print(rphst_adj['idx_transactions'].sum() == spoc_adj['idx_transactions'].sum()== sml_adj['idx_transactions'].sum())


# Output

In [ ]:
bq_output = spoc_adj[['date_time','index']]

In [ ]:
#saves dataframe as big query table
#to replace dataframe, change if_exists = 'replace'
bq_output.to_gbq('ons-fintrans-analysis-prod.fin_wip_notebook.cardholder_adjustments_april2023', project_id=None, chunksize=None, reauth=False, 
                 if_exists='replace', auth_local_webserver=True, table_schema=None, 
                 location=None, progress_bar=True, credentials=None)